# Crello analysis

This notebook qualitatively analyzes learned models in crello-document dataset.

In [ ]:
import itertools
import json
import logging
import sys
import os

import tensorflow as tf
from IPython.display import display, HTML

sys.path.append('../src/canvas-vae')

from canvasvae.models.vae import VAE
from canvasvae.data import DataSpec
from canvasvae.helpers.svg import SVGBuilder
from canvasvae.helpers.retrieve import ImageRetriever

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

Load datasets

In [ ]:
dataset_name = 'crello-document'
dataspec = DataSpec(dataset_name, f"../data/{dataset_name}", batch_size=2)
train_dataset = dataspec.make_dataset('train', shuffle=True)
test_dataset = dataspec.make_dataset('test', shuffle=True)

Load learned models. Edit the job id list to choose which experiment to inspect.

In [ ]:
def load_model(job_id, best_or_final='final'):
    job_dir = f"../tmp/jobs/canvasvae/{job_id}"
    with open(os.path.join(job_dir, 'test_results.json'), 'r') as f:
        results = json.load(f)
        
    name = f"{results['decoder_type']}_{results['block_type']}{results['num_blocks']}"
    model = VAE(
        dataspec.make_input_columns(),
        decoder_type=results['decoder_type'],
        latent_dim=results['latent_dim'],
        num_blocks=results['num_blocks'],
        block_type=results['block_type'],
    )
    model.compile(optimizer='adam')
    model.load_weights(f"{job_dir}/checkpoints/{best_or_final}.ckpt")
    return name, model


# Fill in appropriate job ids.
job_ids = ['20210827035436']
models = {name: model for name, model in map(load_model, job_ids)}
models

Build the image retrieval subsystem for visualization. This can take long (~15min) depending on the available CPU/GPUs.

In [ ]:
encoder = tf.keras.models.load_model('../data/pixelvae/encoder')

train_image_db = ImageRetriever(
    dataset_path='../data/crello-image/train*',
    cache_dir='../tmp/retriever/train_images',
)
%time train_image_db.build(encoder)

test_image_db = ImageRetriever(
    dataset_path='../data/crello-image/test*',
    cache_dir='../tmp/retriever/test_images',
)
%time test_image_db.build(encoder)

Prepare SVG document builders for visualization

In [ ]:
builder0 = SVGBuilder(
    max_width=128, 
    max_height=192,
    key='type',
    preprocessor=dataspec.preprocessor,
)
builder1 = SVGBuilder(
    max_width=128, 
    max_height=192,
    key='color',
    preprocessor=dataspec.preprocessor,
    image_db=train_image_db,
    render_text=True,
)
builder2 = SVGBuilder(
    max_width=128, 
    max_height=192,
    key='color',
    preprocessor=dataspec.preprocessor,
    image_db=test_image_db,
    render_text=True,
)

### Reconstruction and sampling

The following visualizes reconstruction with stochastic sampling

In [ ]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)


def visualize_reconstruction(model, example, dataspec, input_builders, output_builders):
    svgs = []
    for builder in input_builders:
        svgs.append(list(map(builder, dataspec.unbatch(example))))
    for i in range(3):
        prediction = model(example, sampling=(i > 0))
        for builder in output_builders:
            svgs.append(list(map(builder, dataspec.unbatch(prediction))))
    return [list(grouper(row, len(input_builders))) for row in zip(*svgs)] 


example = next(iter(test_dataset.take(1)))
svgs = {}
for key, model in models.items():
    print(key)
    svgs[key] = visualize_reconstruction(model, example, dataspec, [builder0, builder2], [builder0, builder1])
    for row in svgs[key]:
        display(HTML('<div>%s</div>' % ' '.join(itertools.chain.from_iterable(row))))

### Interpolation

Show interpolation between two samples.

In [ ]:
def visualize_interpolation(model, example1, example2, dataspec, input_builders, output_builders, num_samples=8):
    z1 = model.encoder(example1, sampling=False)
    z2 = model.encoder(example2, sampling=False)
    cols = []
    for builder in input_builders:
        cols.append(list(map(builder, dataspec.unbatch(example1))))
    for u in tf.cast(tf.linspace(0, 1, num_samples), tf.float32):
        z = z1 * (1 - u) + z2 * u
        prediction = model.decoder(z)
        for builder in output_builders:
            cols.append(list(map(builder, dataspec.unbatch(prediction))))
    for builder in input_builders:
        cols.append(list(map(builder, dataspec.unbatch(example2))))
        
    return [list(grouper(row, len(input_builders))) for row in zip(*cols)]


itr = iter(test_dataset.take(2))
example1, example2 = next(itr), next(itr)

layouts, previews = {}, {}
for key, model in models.items():
    print(key)
    layouts[key] = visualize_interpolation(model, example1, example2, dataspec, [builder0], [builder0])
    previews[key] = visualize_interpolation(model, example1, example2, dataspec, [builder2], [builder1])
    for row1, row2 in zip(layouts[key], previews[key]):
        display(HTML('<div>%s</div>' % ' '.join(itertools.chain.from_iterable(row1))))
        display(HTML('<div>%s</div>' % ' '.join(itertools.chain.from_iterable(row2))))

### Random generation

Show randomly generated documents.

In [ ]:
def generate_random(model, builders, batch_size=16):
    z = tf.random.normal(shape=(batch_size, model.decoder.latent_dim))
    prediction = model.decoder(z)
    svgs = []
    for item in dataspec.unbatch(prediction):
        svgs.append(tuple(builder(item) for builder in builders))
    return svgs

for key, model in models.items():
    print(key)
    generated = generate_random(model, [builder0, builder1])
    display(HTML('<div>%s</div>' % ' '.join(itertools.chain.from_iterable(generated))))